In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import ImageEnhance, Image

In [5]:
img = cv2.imread("./Imagenes/7.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

#########

# noise removal
kernel = np.ones((3,3),np.uint8)/9
opening = cv2.morphologyEx(thresh,cv2.MORPH_BLACKHAT,kernel, iterations = 2)
# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.2*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0
markers = cv2.watershed(img, markers)
img[markers == -1] = [0, 0, 255]
cv2.imshow("img", cv2.resize(img, (900,700)))
cv2.waitKey(0)
cv2.destroyAllWindows()

# Código que funciona re bien

### Codigo base

In [62]:
img2 = cv2.imread("./Imagenes/51.jpg")
grises = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
_,th =  cv2.threshold(grises, 120, 255, cv2.THRESH_BINARY)

cnts,_ = cv2.findContours(th, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 
contornos = cv2.drawContours(img2, cnts, -1, (255,0,0),2)
cv2.imshow("img", cv2.resize(contornos, (700,800)))
cv2.waitKey(0)
cv2.destroyAllWindows()
len(cnts)


24253

In [127]:
img2 = cv2.imread("./Imagenes/Ps1.png")[480:1500]
grises = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
_,th =  cv2.threshold(grises, 70, 255, cv2.THRESH_BINARY)

cnts,_ = cv2.findContours(th, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 
contornos = cv2.drawContours(img2, cnts, -1, (255,0,0),2)
cv2.imshow("img", cv2.resize(contornos, (500, 700)))
cv2.waitKey(0)
cv2.destroyAllWindows()
len(cnts)

128

## Selección de contornos super duper buena

### Imagenes sin editar

In [91]:
img2 = cv2.imread("./Imagenes/Ps4.png")[480:1500] # Importa la imagen
th = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) # Convierte a escala de grises
_,th =  cv2.threshold(th, 70, 255, cv2.THRESH_BINARY)
cv2.imshow("img", cv2.resize(th, (500, 700)))

bkg = cv2.imread("./Imagenes/bkg.png")[480:1500]
bkg = cv2.cvtColor(bkg, cv2.COLOR_BGR2GRAY)
_,bkg =  cv2.threshold(bkg, 30, 255, cv2.THRESH_BINARY)

trace = cv2.subtract(th,bkg) # Se quita el fondo de la imagen para disminuir ruido

cnts,_ = cv2.findContours(trace, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) # Halla los contornos externos
CNTS = ()
for i in range(0, len(cnts)): # Quita algunos de los contornos pequeños que corresponden a ruido
    if cnts[i].shape[0] > 60:
        CNTS += (cnts[i], )

contornos = cv2.drawContours(img2, CNTS, -1, (0,0,255),2) # Dibuja los contornos
contornos = cv2.putText(contornos, "Particulas (aprox): {}".format(len(CNTS)), (50,70), cv2.FONT_HERSHEY_DUPLEX, 1.4, (200,82,100), 5)
cv2.imshow("img", cv2.resize(contornos, (500, 700)))

cv2.waitKey(0)
cv2.destroyAllWindows()
len(CNTS)

3

### Imagenes con edicion en programa externo

In [92]:
img2 = cv2.imread("./Imagenes/Ps4_v2.jpeg")[480:1500] # Importa la imagen
th = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) # Convierte a escala de grises
_,th =  cv2.threshold(th, 70, 255, cv2.THRESH_BINARY)
cv2.imshow("img", cv2.resize(th, (500, 700)))

bkg = cv2.imread("./Imagenes/bkg_v2.jpg")[480:1500]
bkg = cv2.cvtColor(bkg, cv2.COLOR_BGR2GRAY)
_,bkg =  cv2.threshold(bkg, 5, 255, cv2.THRESH_BINARY)

trace = cv2.subtract(th,bkg) # Se quita el fondo de la imagen para disminuir ruido

cnts,_ = cv2.findContours(trace, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) # Halla los contornos externos
CNTS = ()
for i in range(0, len(cnts)): # Quita algunos de los contornos pequeños que corresponden a ruido
    if cnts[i].shape[0] > 60:
        CNTS += (cnts[i], )

contornos = cv2.drawContours(img2, CNTS, -1, (0,0,255),2) # Dibuja los contornos
cv2.imshow("img", cv2.resize(contornos, (500, 700)))

cv2.waitKey(0)
cv2.destroyAllWindows()
len(CNTS)

6

### Imagenes con edición en python

In [21]:
imgO = cv2.imread("./Imagenes/Ps4.png")[480:1500] # Importa la imagen
img = cv2.cvtColor(imgO, cv2.COLOR_BGR2GRAY) #Convierte a RGB
img = Image.fromarray(img) #Convierte a imagen de PIL

#Aplica los filtros a la imagen
img = ImageEnhance.Brightness(img).enhance(0.34)
img = ImageEnhance.Contrast(img).enhance(4)
img = ImageEnhance.Sharpness(img).enhance(4)
img = np.array(img)


bkg = cv2.imread("./Imagenes/bkg.png")[480:1500] # Importa la imagen
bkg = cv2.cvtColor(bkg, cv2.COLOR_BGR2RGB) #Convierte a RGB
bkg = cv2.cvtColor(bkg, cv2.COLOR_RGB2GRAY) #Convierte a grises

bkg = Image.fromarray(bkg) #Convierte a imagen de PIL

#Aplica los filtros al background
bkg = ImageEnhance.Brightness(bkg).enhance(0.34)
bkg = ImageEnhance.Contrast(bkg).enhance(5)
bkg = ImageEnhance.Sharpness(bkg).enhance(2)
bkg = np.array(bkg)

#Convierte a imagen B/N
_,th =  cv2.threshold(img, 30, 255, cv2.THRESH_BINARY)
_,bkg =  cv2.threshold(bkg, 10, 255, cv2.THRESH_BINARY)

trace = cv2.subtract(th,bkg) # Se quita el fondo de la imagen para disminuir ruido

cnts,_ = cv2.findContours(trace, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) # Halla los contornos externos
CNTS = ()
for i in range(0, len(cnts)): # Quita algunos de los contornos pequeños que corresponden a ruido
    if cnts[i].shape[0] > 70:
        CNTS += (cnts[i], )

contornos = cv2.drawContours(imgO.copy(), CNTS, -1, (0,0,255),2) # Dibuja los contornos
contornos = cv2.putText(contornos, "Particulas (aprox): {}".format(len(CNTS)), (50,70), cv2.FONT_HERSHEY_DUPLEX, 1.4, (200,82,150), 5) #Coloca la cantidad de particulas en cada imagen

imgO = cv2.putText(imgO.copy(), "Imagen original", (50,70), cv2.FONT_HERSHEY_DUPLEX, 1.4, (200,82,150), 5)

frame = cv2.hconcat([imgO, contornos])

cv2.imshow("img", cv2.resize(frame, (510, 540)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
frame.shape

(1020, 2160, 3)

### Video

In [2]:
#Background
bkg = cv2.imread("./Imagenes/bkg.png")[480:1500] # Importa la imagen
bkg = cv2.cvtColor(bkg, cv2.COLOR_BGR2RGB) #Convierte a RGB
bkg = cv2.cvtColor(bkg, cv2.COLOR_RGB2GRAY) #Convierte a grises

bkg = Image.fromarray(bkg) #Convierte a imagen de PIL

#Aplica los filtros al background
bkg = ImageEnhance.Brightness(bkg).enhance(0.34)
bkg = ImageEnhance.Contrast(bkg).enhance(5)
bkg = ImageEnhance.Sharpness(bkg).enhance(2)
bkg = np.array(bkg)

#Convierte a imagen B/N
_,bkg =  cv2.threshold(bkg, 10, 255, cv2.THRESH_BINARY)

video = cv2.VideoCapture("../Camara de niebla/TOMA 2/Video_up_view.mp4")
newVideo = cv2.VideoWriter('tracesVideoFull.avi', cv2.VideoWriter_fourcc(*"mp4v"), 30, (2160, 1020)) #Variable para el nuevo video

j=0
while video.isOpened():
    ret, frame = video.read()
    if ret:
        frame = cv2.rotate(frame, cv2.ROTATE_180)
        imgO = frame[480:1500] # Recorta el frame
        img = cv2.cvtColor(imgO, cv2.COLOR_BGR2GRAY) #Convierte a RGB
        img = Image.fromarray(img) #Convierte a imagen de PIL

        #Aplica los filtros a la imagen
        img = ImageEnhance.Brightness(img).enhance(0.34)
        img = ImageEnhance.Contrast(img).enhance(4)
        img = ImageEnhance.Sharpness(img).enhance(4)
        img = np.array(img)

        #Convierte a imagen B/N
        _,th =  cv2.threshold(img, 30, 255, cv2.THRESH_BINARY)

        trace = cv2.subtract(th,bkg) # Se quita el fondo de la imagen para disminuir ruido

        cnts,_ = cv2.findContours(trace, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) # Halla los contornos externos
        CNTS = ()
        for i in range(0, len(cnts)): # Quita algunos de los contornos pequeños que corresponden a ruido
            if cnts[i].shape[0] > 70:
                CNTS += (cnts[i], )

        contornos = cv2.drawContours(imgO.copy(), CNTS, -1, (0,0,255), 2) # Dibuja los contornos
        contornos = cv2.putText(contornos, "Particulas (aprox): {}".format(len(CNTS)), (50,70), cv2.FONT_HERSHEY_DUPLEX, 1.4, (200,82,150), 5) #Coloca la cantidad de particulas en cada imagen

        imgO = cv2.putText(imgO.copy(), "Imagen original", (50,70), cv2.FONT_HERSHEY_DUPLEX, 1.4, (200,82,150), 5)

        newFrame = cv2.hconcat([imgO, contornos])
        newVideo.write(newFrame)

    else:
        break
    # j+=1
    # if j == 400:
    #     break
 
video.release()
newVideo.release()
cv2.destroyAllWindows()
